# Auto-registering [`IPython`] magics

[`IPython`] uses the concepts of [magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html), which, as the name implies, seem like magic if one does not know how they work. In this post we will be trying to push the boundaries a little by creating our own [`IPython`] magic that is able to auto-register new magics without the user needing to explicitly do so. 

In particular we want to register all functions defined in a cell that match the signature of a magic by invoking a cell magic that we are about to define. This new cell magic needs to be able to perform the following three steps:

1. It needs to extract comprised functions from the cell content.
2. It needs to identify the corresponding magic type for each given function.
3. It needs to register the correct [`IPython`] magic based on the type.

[`IPython`]: https://ipython.org/

We start of by importing everything we need. Apart from [`IPython`] we will use [`enum`](https://docs.python.org/3/library/enum.html) to differentiate between the magic types and [`inspect`](https://docs.python.org/3/library/inspect.html) to examine live objects such as the the functions to be registered.

[`IPython`]: https://ipython.org/

In [1]:
import enum
import inspect
from typing import Callable, Dict, Optional

from IPython.core.magic import register_line_magic, register_cell_magic, register_line_cell_magic

## Extract functions from the cell content

In the first step, we need to extract comprised functions from the cell content. Since the cell content is available as `str`'ing, we [`exec`](https://docs.python.org/3/library/functions.html#exec)'ute it. We are only interested in functions, so we use [`inspect.isfunction()`](https://docs.python.org/3/library/inspect.html#inspect.isfunction) to filter them out.

In [4]:
def get_public_functions(cell: str) -> Dict[str, Callable]:
    content = {}
    exec(cell, None, content)
    return {name: fn for name, fn in content.items() if inspect.isfunction(fn)}

We can test this with a simple example:

In [5]:
cell = """
foo = None

def bar():
    return "baz"
"""

`get_public_functions(cell)` should return a function `bar` that returns the `str`'ing `"baz"` when called. The variable `foo` should not be returned, since it is not a function.

In [6]:
public_functions = get_public_functions(cell)

assert "foo" not in public_functions
assert "bar" in public_functions

bar = public_functions["bar"]
assert bar() == "baz"

## Identify the magic type

In the second step we need to identify the corresponding magic type of a given function. Before we get to that, we first define an [`enum.Enum`](https://docs.python.org/3/library/enum.html?highlight=enum#enum.Enum) that makes it easy to work with multiple discrete values.

In [10]:
class MagicType(enum.Enum):
    LINE = enum.auto()
    CELL = enum.auto()
    LINE_CELL = enum.auto()

To differentiate between the different magic types we need to look at their signature:

- Line magics take a single argument `line` without a default.
- Cell magics take two arguments `line, cell` without defaults.
- Line/cell magics take two arguments `line, cell=None` where the second defaults to `None`.

These rules can be implemented straight forward with [`inspect.getfullargspec()`](https://docs.python.org/3/library/inspect.html#inspect.getfullargspec) that lets us examine the signature of functions.

In [11]:
def detect_magic_type(fn: Callable) -> Optional[MagicType]:
    spec = inspect.getfullargspec(fn)
    if len(spec.args) == 1:
        if spec.defaults is None:
            return MagicType.LINE
    elif len(spec.args) == 2:
        if spec.defaults is None:
            return MagicType.CELL
        elif len(spec.defaults) == 1 and spec.defaults[-1] is None:
            return MagicType.LINE_CELL
        
    # The function does not fit the IPython magic scheme
    return None

Lets also test this with a simple example:

In [12]:
def my_line_magic(line):
    pass

def my_cell_magic(line, cell):
    pass

def my_line_cell_magic(line, cell=None):
    pass

def my_other_function():
    pass

`detect_magic_type` should return the correct `MagicType` for `my_line_magic`, `my_cell_magic`, and `my_line_cell_magic`. At the same time, it should return `None` for `my_other_function`, since its signature does not fit the scheme.

In [13]:
assert detect_magic_type(my_line_magic) == MagicType.LINE
assert detect_magic_type(my_cell_magic) == MagicType.CELL
assert detect_magic_type(my_line_cell_magic) == MagicType.LINE_CELL
assert detect_magic_type(my_other_function) is None

## Register a function as magic

In [ ]:
In the third step, we need to register a function as [`IPython`] magic based on its `MagicType`.

In [16]:
registrars = {
    MagicType.LINE: register_line_magic,
    MagicType.CELL: register_cell_magic,
    MagicType.LINE_CELL: register_line_cell_magic,
}

def register_magic(fn: Callable, magic_type: MagicType) -> None:
    try:
        registrars[magic_type](fn)
    except KeyError as error:
        raise RuntimeError(f"Unknown magic type {magic_type}") from error

Lets put this to a test:

In [18]:
def my_line_magic(line):
    print("This is my line magic!")

Calling `register_magic` with `my_line_magic` should give us access to the `%my_line_magic` magic that prints `"This is my line magic!"` when used.

In [19]:
register_magic(my_line_magic, MagicType.LINE)

%my_line_magic

This is my line magic!


## Putting everything together

Finally, we can put every piece together by creating a custom cell magic that performs the three steps for us.

In [ ]:
@register_cell_magic
def auto_register_magics(line: str, cell: str) -> None:
    public_functions = get_public_functions(cell)
    
    for name, fn in public_functions.items():
        magic_type = detect_magic_type(fn)
        if not magic_type:
            continue
            
        register_magic(fn, magic_type)

Invoking the `%%auto_register_magics` cell magic should now automatically register all comprised functions as magic with the correct type:

In [22]:
%%auto_register_magics

def my_second_line_magic(line):
    print("This is my second line magic!")

def my_cell_magic(line, cell):
    print("This is my cell magic!")

def my_line_cell_magic(line, cell=None):
    print(f"This is my line/cell magic! It was used as {'line' if cell is None else 'cell'} magic!")

Lets see if it worked:

In [24]:
%my_second_line_magic

This is my second line magic!


In [51]:
%%my_cell_magic

pass

This is my cell magic!


In [52]:
%my_line_cell_magic

This is my line/cell magic! It was used as line magic!


In [53]:
%%my_line_cell_magic

pass

This is my line/cell magic! It was used as cell magic!


Yay, it worked!

## Conclusion

This notebook showcases how to push the boundaries of [`IPython`] magics, by showcasing how a magic can automatically create new magics. Of course the approach used here is a proof-of-concept and not robust enough for wide-spread used.